In [ ]:
# 1. Import necessary libraries
import pandas as pd
import numpy as np
import yfinance as yf
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
from tensorflow import keras
from keras import layers, initializers 
from keras_tuner import HyperModel, BayesianOptimization
from pypfopt import EfficientFrontier, risk_models, expected_returns
import json
import re
import pickle
import  os

In [ ]:
# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)
os.environ['PYTHONHASHSEED'] = '42'
os.environ['TF_DETERMINISTIC_OPS'] = '1'

In [ ]:
# with open('symbols.json', 'r') as f:
#    tickers = json.load(f)

# pk_filenames.json = ["data/ibm.pk1", "data/aapl.pk1"]

with open('pk_filenames.json','r') as f:
    filenames = json.load(f)

# for filename in filenames:
#     df = pd.read_pickle(filename)

In [ ]:
# 2. Data Collection
# with open('symbols.json', 'r') as f:
#    tickers = json.load(f)

# def fetch_data(ticker, start, end):
#     data = yf.download(ticker, start=start, end=end)
#     return data['Adj Close']

# tickers = ['AAPL', 'MSFT', 'GOOGL', 'TSLA', 'NVDA', 'TLT', 'COST', 'WMT', 'BA', 'DIS', 'JPM', 'AMD']
all_expected_returns = {}
all_data = pd.DataFrame()

In [ ]:
# Function to aggregate daily returns into weekly returns
def aggregate_returns(data, freq='W'):
    return data.resample(freq).ffill().pct_change().dropna()

In [ ]:
# Walk-forward validation function
def walk_forward_validation(data, model, time_step, n_test):
    predictions = []
    train, test = data[:-n_test], data[-n_test:]
    for i in range(n_test):
        train_set = pd.concat([train, test[:i]])
        X_train, y_train = create_dataset(train_set, time_step)
        model.fit(X_train, y_train, epochs=10, verbose=0)
        input_data = train_set[-time_step:].values.reshape((1, time_step, 1))
        yhat = model.predict(input_data, verbose=0)
        predictions.append(yhat[0, 0])
    return predictions

In [ ]:
# Create dataset function to prepare the data for LSTM
def create_dataset(data, time_step):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:i + time_step])
        y.append(data[i + time_step])
    return np.array(X), np.array(y)

In [ ]:
for filename in filenames:
    df = pd.read_pickle(filename)
    # data = fetch_data(ticker, '2020-01-01', '2023-01-01')
    data = df['Adj Close']
    ticker = filename.split('/')[1].split('.')[0]

    # all_data[ticker] = data  # Store data for covariance calculation
    

    # Data Preprocessing
    weekly_returns = aggregate_returns(data)  # Aggregate to weekly returns

    # Normalize the weekly returns using Min-Max Scaler
    scaler = MinMaxScaler(feature_range=(0, 1))
    weekly_returns_reshaped = weekly_returns.values.reshape(-1, 1)
    scaler.fit(weekly_returns_reshaped)
    weekly_returns_normalized = scaler.transform(weekly_returns_reshaped)

    # Reshape data for LSTM in a compatible sliding window format
    time_step = 4
    X, y = create_dataset(weekly_returns_normalized, time_step)

    # LSTM Modeling with fixed initializers
    class LSTMHyperModel(HyperModel):
        def build(self, hp):
            model = keras.Sequential()
            model.add(layers.Input(shape=(time_step, 1)))  # Update input shape
            model.add(layers.LSTM(
                units=hp.Int('units', min_value=32, max_value=128, step=32),
                activation='relu',
                kernel_initializer=initializers.GlorotUniform(seed=42),  # Fixed seed for weights
                bias_initializer=initializers.Zeros()  # Fixed bias initializer
            ))
            model.add(layers.Dense(1))  # Ensure the output layer has a fixed size
            model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3])), loss='mse')
            return model

    # Hyperparameter tuning
    tuner = BayesianOptimization(
        LSTMHyperModel(),
        objective='val_loss',
        max_trials=2,
        executions_per_trial=1,
        directory='lstm_tuning',
        project_name=f'portfolio_optimization_{ticker}'
    )
    # Search for the best hyperparameters
    tuner.search(X, y, epochs=10, validation_split=0.2)
    
    # Use walk-forward validation to evaluate the model
    n_test = 52  # Number of weeks to predict
    best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]
    model = tuner.hypermodel.build(best_hps)
    predicted_returns_normalized = walk_forward_validation(pd.Series(weekly_returns_normalized.flatten()), model, time_step, n_test)

    # Inverse transform the predicted returns
    predicted_returns = scaler.inverse_transform(np.array(predicted_returns_normalized).reshape(-1, 1)).flatten()

    # Calculate the total return over the 52 weeks using compounded returns
    compounded_return = np.prod(1 + np.array(predicted_returns)) - 1

    # Use the compounded return as the annualized expected return
    annualized_return = compounded_return
    all_expected_returns[ticker] = annualized_return
    all_data[ticker] = weekly_returns  # Add this line to populate the all_data dictionary

    # Print annualized expected returns for each ticker
    print(f"Annualized Expected Returns for {ticker}: {annualized_return}")

    filename = ticker + '_annualized_return.pkl'
    with open(filename,'wb') as f:
        pickle.dump(annualized_return,f)

In [ ]:
# Convert the dictionary to a Pandas Series
expected_returns_series = pd.Series(all_expected_returns)

In [ ]:
cov_matrix = risk_models.risk_matrix(all_data, returns_data=True, method='ledoit_wolf')

In [ ]:
# Portfolio Optimization
# Calculate the covariance matrix using all tickers' data
# cov_matrix = risk_models.risk_matrix(returns_df, method='ledoit_wolf' )
# https://pyportfolioopt.readthedocs.io/en/latest/RiskModels.html
ef = EfficientFrontier(expected_returns=expected_returns_series, cov_matrix=cov_matrix)
weights = ef.max_sharpe()
cleaned_weights = ef.clean_weights()

# Print the optimized portfolio weights
print("Optimized Portfolio Weights:", cleaned_weights)

In [ ]:
with open('cleaned_weights.pkl','wb') as f:
    pickle.dump(cleaned_weights,f)

In [ ]:
# VISUALIZATIONS

# Dependencies and Libraries
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pickle
from pypfopt import expected_returns, EfficientSemivariance
from pypfopt.base_optimizer import BaseConvexOptimizer
from pypfopt.objective_functions import ex_post_tracking_error
from pypfopt import plotting
from pypfopt import risk_models

In [ ]:
# Cleaned Weights and Expected Returns Series
with open('cleaned_weights.pkl', 'rb') as f:
    cleaned_weights = pickle.load(f)

with open('expected_returns_series.pkl', 'rb') as f:
    expected_returns_series = pickle.load(f)

In [ ]:
# Efficient Frontier
ef = EfficientFrontier(mu, S, weight_bounds=(None, None))
ef.add_constraint(lambda w: w[0] >= 0.2)
ef.add_constraint(lambda w: w[2] == 0.15)
ef.add_constraint(lambda w: w[3] + w[4] <= 0.10)

fig, ax = plt.subplots()
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=True)
plt.show()

# plt.figure(figsize=(10, 6))
# ax = plotting.plot_efficient_frontier(ef, show_assets=True)
# plt.title('Efficient Frontier')
# plt.xlabel('Volatility (Standard Deviation)')
# plt.ylabel('Expected Return')
# plt.grid(True)
# plt.show()

In [ ]:
# **OPTIONAL (EF): Passing a range of parameters (risk, utility, or returns) to generate a frontier**

# 100 portfolios with risks between 0.10 and 0.30
risk_range = np.linspace(0.10, 0.40, 100)
plotting.plot_efficient_frontier(ef, ef_param="risk", ef_param_range=risk_range,
                                show_assets=True, showfig=True)

In [ ]:
# **OPTIONAL (EF): Generating more complex plots**

# Plotting both the efficient frontier and randomly generated (suboptimal) portfolios, ft. Sharpe ratio:
fig, ax = plt.subplots()
ef_max_sharpe = ef.deepcopy()
plotting.plot_efficient_frontier(ef, ax=ax, show_assets=False)

# Finding Tangency Portfolio
ef_max_sharpe.max_sharpe()
ret_tangent, std_tangent, _ = ef_max_sharpe.portfolio_performance()
ax.scatter(std_tangent, ret_tangent, marker="*", s=100, c="r", label="Max Sharpe")

# Generating Random Portfolios
n_samples = 10000
w = np.random.dirichlet(np.ones(ef.n_assets), n_samples)
rets = w.dot(ef.expected_returns)
stds = np.sqrt(np.diag(w @ ef.cov_matrix @ w.T))
sharpes = rets / stds
ax.scatter(stds, rets, marker=".", c=sharpes, cmap="viridis_r")

# Output
ax.set_title("Efficient Frontier with random portfolios")
ax.legend()
plt.tight_layout()
plt.savefig("ef_scatter.png", dpi=200)
plt.show()

In [ ]:
# Optimized Portfolio Weights
plt.figure(figsize=(10, 6))
sns.barplot(x=list(cleaned_weights.keys()), y=list(cleaned_weights.values()))
plt.title('Optimized Portfolio Weights')
plt.xlabel('Assets')
plt.ylabel('Weights')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Expected Annualized Returns
plt.figure(figsize=(10, 6))
sns.barplot(x=expected_returns_series.index, y=expected_returns_series.values)
plt.title('Expected Annualized Returns')
plt.xlabel('Assets')
plt.ylabel('Expected Return')
plt.xticks(rotation=45)
plt.grid(True)
plt.show()

In [ ]:
# Cumulative Portfolio Returns
def calculate_portfolio_cumulative_returns(weights, returns_df):
    portfolio_returns = (returns_df * weights).sum(axis=1)
    cumulative_returns = (1 + portfolio_returns).cumprod() - 1
    return cumulative_returns

cumulative_returns = calculate_portfolio_cumulative_returns(cleaned_weights, all_data)

plt.figure(figsize=(10, 6))
plt.plot(cumulative_returns, label='Portfolio')
plt.title('Cumulative Returns of Optimized Portfolio')
plt.xlabel('Date')
plt.ylabel('Cumulative Return')
plt.grid(True)
plt.legend()
plt.show()

In [ ]:
# Covariance Matrix
cov_matrix = risk_models.risk_matrix(all_data, returns_data=True, method='ledoit_wolf')

# Converting the covariance matrix to a DataFrame for better visualization
cov_matrix_df = pd.DataFrame(cov_matrix, index=all_data.columns, columns=all_data.columns)

# Heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(cov_matrix_df, annot=True, fmt=".2f", cmap='coolwarm', cbar_kws={'label': 'Covariance'})
plt.title('Covariance Matrix Heatmap')
plt.xticks(rotation=45)
plt.yticks(rotation=45)
plt.tight_layout()
plt.show()